### Twitter Analysis Part 3

Now that I have a list of the Twitter users that follow Lawson, the following code retrieves all of the users they follow, counts how many Lawson followers also follow them, and then calculates the ratio of that count to the number of people that follow the user. 

In [ ]:
working_dir = "C:\\Users\\Mary Makris\\Documents\\Applied Data Analytics\\APIs\\"
outpath = "C:\\Users\\Mary Makris\\Documents\\Applied Data Analytics\\Projects\\Twitter\\"

from datetime import datetime
import json
from twitter_functions import *
from collections import Counter
from collections import defaultdict

In [ ]:
#create dictionary of twitter credientials and then access the Twitter API
auth =  { "consumer_key": "insert",
          "consumer_secret": "insert",
          "access_key": "insert",
          "access_secret": "insert"
        }

api = initialize_twitter(auth)

Here I initialize an empty list and then add all of Lawson's followers to it

In [ ]:
followers_list = list()

with open(working_dir + "Lawson_Followers.csv", "r", encoding = "UTF-8") as ffile:
    next(ffile)
    for idx, line in enumerate(ffile):
        line = line.strip()
        followers_list.append(line)

Now I am going to go lookup the full information on Lawsons followers

In [ ]:
##initialize a list
friends_user_ids = []

# All records will be a dictionary with the twitter ID as the key and 
# a UserRecord as the value. 
all_records = lookup_users_from_handles(api, followers_list)

# This puts the ids we are getting followers from in a list. 
for id in all_records : #access the keys, which are ids.
    friends_user_ids.append(id)

In [ ]:
# How long is it going to take us to pull these friends?
total_friends = 0
for id, rec in all_records.items() :
    total_friends += rec.friends_count
    
print("A complete run with no limits run is " + 
      "going to take {min:.2f} minutes ({hour:.2f} hours)".format(min=total_friends/5000,
                                                                              hour=total_friends/(60*5000)))

In [ ]:
# Now I'm going to pull all the friends of the people in our list
#
followers_of_starting = gather_friends(api,
                                         friends_user_ids,
                                         friends_limit=None)   

# followers_of_starting will be a dictionary with the key being the id(s) in starting_user_id
# and the value is a list of all the followers.
#check a couple key value pairs out
first2pairs = {k: followers_of_starting[k] for k in list(followers_of_starting)[:2]}
print(first2pairs)

Now that we have a list of all the friends IDs, I am going to append them all into one big list and then use a counter dictionary to count how many time an ID reoccurs. 

In [ ]:
#initialize a list
friends_ids = []

#make a list of the ids in the dictionary
friends_ids_list = list(followers_of_starting.values())

#it's a list of a list, so I needed to dig one level deeper
for sublist in friends_ids_list:
    for item in sublist:
        friends_ids.append(item)
        
#Now I can count them all up!        
friend_count = Counter(friends_ids) 

##make sure it worked
first2counts = {k: friend_count[k] for k in list(friend_count)[:2]}
print(first2counts)

In [ ]:
for start_id, list_of_followers in followers_of_starting.items() :
    
    # Using a set here instead of a list so that we pull each ID only once.
    ids_to_hydrate = {id for id in list_of_followers if id not in all_records} 
    
    these_records = lookup_users_from_ids(api,ids=ids_to_hydrate)

    for id, rec in these_records.items() :
        all_records[id] = rec

In [ ]:
#create new dictionary with all_records dictionary fields I need and the ID counter count
finalrecords_dict = defaultdict(list)

for id, rec in all_records.items():
    if rec.followers_count > 200 and rec.followers_count < 5000:
        finalrecords_dict[id] = rec.name, rec.screen_name, rec.followers_count, friend_count[id],(friend_count[id]/rec.followers_count)

In [ ]:
##And now we can write out all the records!
with open(outpath + "finalrecords1.csv",'w',encoding="utf8") as donefile:
    headers = ['id', 'name', 'screen name', 'followers_count', 'count', 'affinity_score']          
    donefile.write(",".join(headers) + "\n")
    for key,value in finalrecords_dict.items():
        record = [str(thing) for thing in value]
        donefile.write (str(key) +"," + ",".join(record) + "\n")

In [ ]:
##for some reason even though I added this to the other Twitter Functions you wrote, Jup Note couldn't find it so I 
##added it here too

def gather_friends(api, ids, friends_limit = 10000, file_name=None) :
    ''' from a set of IDs, get all the people they follow's ids up to follower
        limit. Returns a dictionary with key = id, value = following_id list
    '''
    
    ret_dict = defaultdict(list)    
    
    for this_id in ids :
        print("Pulling friends for " + str(this_id))        
        pulled_friends = 0
        try :
            for page in tweepy.Cursor(api.friends_ids,user_id = this_id).pages() :
                pulled_friends += len(page['ids'])
                ret_dict[this_id].extend(page['ids'])
                print("Number pulled: " + str(pulled_friends))                
                
                if friends_limit and pulled_friends > friends_limit :
                    break

            if file_name is not None :
                with open(file_name,'a') as ofile :
                    for friend_id in ret_dict[this_id] :
                        ofile.write("\t".join([str(this_id),str(friend_id)]) + "\n")

        except tweepy.TweepError :
            print("Failed to run command on that user (probably protected or bad id). Skipping...")
            
    return(ret_dict)